`data/propeller_segmented.png`:

![](data/propeller_segmented.png)

In [1]:
#!/usr/bin/env python3
"""
Compute total blade projected area and AEA0 from a color-segmented propeller image.

Assumptions for the image:
- Blades are GREEN
- Hub is RED
- Background is BLUE

You must provide:
- IMAGE_PATH: path to your segmented image
- PROP_DIAMETER_CM: real propeller diameter in centimeters
"""

import numpy as np
from PIL import Image
import math

# --------------------------------------------------------------------
# Configuration
# --------------------------------------------------------------------
IMAGE_PATH = "data/propeller_segmented.png"
PROP_DIAMETER_CM = 9*2.54   # in -> cm
N_BLADES = 3                        # number of blades

# --------------------------------------------------------------------
# Load image and build masks
# --------------------------------------------------------------------
img = Image.open(IMAGE_PATH).convert("RGB")
arr = np.array(img, dtype=np.uint8)

R = arr[:, :, 0].astype(np.int16)
G = arr[:, :, 1].astype(np.int16)
B = arr[:, :, 2].astype(np.int16)

# Simple color thresholds (adjust if your colors differ)
mask_red  = (R > 150) & (G < 100) & (B < 100)   # hub
mask_blue = (B > 150) & (G < 100) & (R < 100)   # background
mask_green = (B > 150) & (G < 100) & (R < 100)   # background

blade_pixels = int(mask_green.sum())

print(f"Blade pixels (total): {blade_pixels}")

# --------------------------------------------------------------------
# Measure prop disk diameter in pixels
# --------------------------------------------------------------------
# Use all blade pixels to estimate disk radius: max distance
coords = np.column_stack(np.where(mask_green))  # (row, col) pairs

if coords.size == 0:
    raise RuntimeError("No blade pixels detected. Check thresholds or image.")

# centroid of blade region
cy, cx = coords.mean(axis=0)

# Euclidean distance of each blade pixel to centroid
dy = coords[:, 0] - cy
dx = coords[:, 1] - cx
radii_px = np.sqrt(dx * dx + dy * dy)

disk_radius_px = radii_px.max()
disk_diameter_px = 2.0 * disk_radius_px

print(f"Estimated disk diameter (pixels): {disk_diameter_px:.2f}")

# --------------------------------------------------------------------
# Convert to real units and compute areas
# --------------------------------------------------------------------
# scale: centimeters per pixel
scale_cm_per_px = PROP_DIAMETER_CM / disk_diameter_px
print(f"Scale: {scale_cm_per_px:.6f} cm/px")

# total blade projected area
blade_area_cm2 = blade_pixels * (scale_cm_per_px ** 2)

# disk area
disk_area_cm2 = math.pi * (PROP_DIAMETER_CM / 2.0) ** 2

# AEA0
AEA0 = blade_area_cm2 / disk_area_cm2

# per-blade area
blade_area_per_blade_cm2 = blade_area_cm2 / N_BLADES

# --------------------------------------------------------------------
# Results
# --------------------------------------------------------------------
print("\n--- Results ---")
print(f"Prop diameter:          {PROP_DIAMETER_CM:.2f} cm")
print(f"Disk area:              {disk_area_cm2:.2f} cm^2")
print(f"Total blade area:       {blade_area_cm2:.2f} cm^2")
print(f"Blade area per blade:   {blade_area_per_blade_cm2:.2f} cm^2")
print(f"AEA0 (expanded area ratio): {AEA0:.3f}")


Blade pixels (total): 109791
Estimated disk diameter (pixels): 759.65
Scale: 0.030093 cm/px

--- Results ---
Prop diameter:          22.86 cm
Disk area:              410.43 cm^2
Total blade area:       99.42 cm^2
Blade area per blade:   33.14 cm^2
AEA0 (expanded area ratio): 0.242


In [2]:
# -------------------------------------------------------------
# CONFIG
# -------------------------------------------------------------
import numpy as np
from PIL import Image
import math

IMAGE_PATH = "data/propeller_segmented.png"
PROP_DIAMETER_CM = 22.86        # 9" prop (used to convert to cm)
ANNULUS_WIDTH = 2.0             # +/- pixel tolerance around 70% radius

# -------------------------------------------------------------
# Load image
# -------------------------------------------------------------
img = np.array(Image.open(IMAGE_PATH).convert("RGB"))
R, G, B = img[:,:,0], img[:,:,1], img[:,:,2]

# -------------------------------------------------------------
# Masks
# -------------------------------------------------------------
mask_red  = (R > 150) & (G < 100) & (B < 100)
mask_blue = (B > 150) & (G < 100) & (R < 100)
mask_blade = ~(mask_red | mask_blue)

# -------------------------------------------------------------
# Hub center
# -------------------------------------------------------------
coords_hub = np.column_stack(np.where(mask_red))
cy, cx = coords_hub.mean(axis=0)

# -------------------------------------------------------------
# Blade pixels in polar coordinates
# -------------------------------------------------------------
coords = np.column_stack(np.where(mask_blade))
y = coords[:,0] - cy
x = coords[:,1] - cx

r = np.sqrt(x*x + y*y)
theta = np.arctan2(y, x)

# Disk radius
R_disk = r.max()
r70 = 0.7 * R_disk

# -------------------------------------------------------------
# Select pixels near r70 (annulus)
# -------------------------------------------------------------
sel = np.abs(r - r70) < ANNULUS_WIDTH
ring_pts = coords[sel]

if ring_pts.shape[0] < 2:
    raise RuntimeError("Not enough pixels found at 70% radius.")

# -------------------------------------------------------------
# Compute chord as straight-line distance between farthest points
# -------------------------------------------------------------
# Convert ring pixels to cartesian (already centered)
x_sel = x[sel]
y_sel = y[sel]

# Two extreme edge points along the blade
# (take max angle separation)
theta_sel = theta[sel]
i_max = np.argmax(theta_sel)
i_min = np.argmin(theta_sel)

x1, y1 = x_sel[i_max], y_sel[i_max]
x2, y2 = x_sel[i_min], y_sel[i_min]

chord_px = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

# -------------------------------------------------------------
# Convert to cm
# -------------------------------------------------------------
disk_diameter_px = 2 * R_disk
scale = PROP_DIAMETER_CM / disk_diameter_px
chord_cm = chord_px * scale

# -------------------------------------------------------------
# Results
# -------------------------------------------------------------
print("R_disk(px):", R_disk)
print("r70(px):", r70)
print("Chord @ 70% radius (px):", chord_px)
print("Chord @ 70% radius (cm):", chord_cm)


R_disk(px): 287.7846417027844
r70(px): 201.4492491919491
Chord @ 70% radius (px): 234.09613409879285
Chord @ 70% radius (cm): 9.297642837773832


In [3]:
import json

results = {
    'prop_diameter_cm': PROP_DIAMETER_CM,
    'disk_area_cm2': float(disk_area_cm2),
    'blade_area_cm2': float(blade_area_cm2),
    'blade_area_per_blade_cm2': float(blade_area_per_blade_cm2),
    'AEA0': float(AEA0),
    'chord_cm': float(chord_cm),
}

with open('propeller_blade_geometry.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, sort_keys=True)

results

{'prop_diameter_cm': 22.86,
 'disk_area_cm2': 410.4330580689732,
 'blade_area_cm2': 99.42354101142746,
 'blade_area_per_blade_cm2': 33.14118033714249,
 'AEA0': 0.24224057749928943,
 'chord_cm': 9.297642837773832}